In [1]:
import pandas as pd
from time import mktime, strptime
import datetime
from tqdm import tqdm

In [2]:
DATABASE_ADDRESS = "../data/Raw/chartevents.csv"
PROCEDURE_EVENTS_ADDRESS = "../data/Raw/procedureevents.csv"
CODE_MAPPING_ADDRESS = "../data/Raw/code_mappings.csv"
CHUNK_SIZE = 1*1000000

###### 20大概需要14分鐘
###### 1需要30.5MB
###### 大檔案有 329,499789行

In [3]:
FILENAME = datetime.datetime.now().strftime('%Y%m%d')
FILENAME

'20230706'

In [4]:
item = dict()
mark = tuple()

In [5]:
df = pd.read_csv(CODE_MAPPING_ADDRESS, engine="python", encoding="unicode_escape")
id_to_label = dict(zip(df["itemid"], df["label"]))
label_to_id = dict(zip(df["label"], df["itemid"]))
# df.head()

In [6]:
mark = (
    "Heart Rate",
    "Respiratory Rate",
    "Non Invasive Blood Pressure systolic",
    "Non Invasive Blood Pressure diastolic",
    "O2 saturation pulseoxymetry",
    "Temperature Celsius"
)

class Patient:
    data = dict()

    def __init__(self):
        self.data = {"subject_id": None, "hadm_id": None, "charttime": None}
        for label_ in mark:
            self.data[label_] = None

    def good(self):
        return sum([self.data[i] != None for i in mark]) >= 5
    
def read_csv_function(road: str) -> pd.core.frame.DataFrame:
    print("Reading", road, "...")
    return pd.read_csv(road, engine='python')

def to_timestamp(s: str):
    try:
        return int(mktime(strptime(s, "%Y-%m-%d %H:%M:%S")))
    except ValueError:
        try:
            return int(mktime(strptime(s, "%Y-%m-%d %H:%M")))
        except ValueError:
            return int(mktime(strptime(s, "%Y/%m/%d %H:%M")))
df = read_csv_function(PROCEDURE_EVENTS_ADDRESS)
dic = {}
for i, it in enumerate(df.itemid):
    if it != 225466:
        continue
    if df.iloc[i].hadm_id not in dic:
        dic[df.iloc[i].hadm_id] = list()
    dic[df.iloc[i].hadm_id].append(to_timestamp(str(df.iloc[i].starttime)))


Reading ../Data/Raw/procedureevents.csv ...


In [7]:
record = []
bar = tqdm(total=330)

  0%|          | 0/330 [00:00<?, ?it/s]

In [8]:
%%time 
reader = pd.read_csv(DATABASE_ADDRESS, usecols=["hadm_id", "charttime", "itemid", "value"], engine="python", encoding="unicode_escape",iterator=True, chunksize=CHUNK_SIZE)
for tlr, df in enumerate(reader):
    bar.update(1)
    df['charttime'] = pd.to_datetime(df['charttime']).round("10min")
    df = df[df["itemid"].isin((220045, 220210, 220179, 220180, 220277, 223762, 223761))]
    output = []
    cnt = 0
    tp = [label_to_id[i] for i in mark]
    for name, group in df.groupby(["hadm_id", "charttime"]):
        patient = Patient()

        for id_, value_ in zip(group.itemid, group.value):
            if id_ == 223761:
                patient.data[223762] = round((float(value_) - 32) * 5 / 9, 1)
                continue
            patient.data[id_to_label[id_]] = value_
        patient.data["hadm_id"], patient.data["charttime"] = name
        output.append(patient.data)
    merged_df = pd.DataFrame(output,columns=['hadm_id','charttime']+list(mark))

    df = merged_df.round(2)

    df.dropna(thresh=5, inplace=True)
    for name, group in df.groupby("hadm_id"):
        group.interpolate(method='bfill', inplace=True)
        group.interpolate(method='ffill', inplace=True)
        df.update(group)

    ans = [False]*len(df)
    for i in range(len(df)):
        if int(df.iloc[i].hadm_id) in dic:
            for j in dic[df.iloc[i].hadm_id]:
                if 0 <= j - to_timestamp(str(df.iloc[i].charttime)) <= 14400:
                    ans[i] = True
                    break

    df["label"] = ans
    df.to_csv(f"../data/Big/chartevents_{FILENAME}_{tlr}_labeled.csv", float_format='%.2f', index=0)

    record.append(df.label.value_counts())
bar.close()

100%|██████████| 330/330 [1:32:13<00:00, 16.77s/it]   

CPU times: user 45min 16s, sys: 22min 10s, total: 1h 7min 26s
Wall time: 1h 32min 13s
